In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [8]:
import logging
import pandas as pd
import numpy as np
import awswrangler as wr
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import datetime
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [4]:
workspace_ids = return_account_ids()
url = return_api_url()

In [9]:
all_audiences = pd.DataFrame()
for workspace in workspace_ids:
    workspace_name = workspace["name"]
    workspace_id = workspace["id"]
    audiences = call_api_with_accountId(
    endpoint_url=f"{url}/audiences/query",
        accountID=workspace_id,
        content={},
        logger=logging
    )
    print(f"{workspace_name} has {len(audiences)} audiences")
    if len(audiences) == 0:
        continue
    audiences = pd.json_normalize(audiences)
    audiences["workspace"] = workspace_name
    all_audiences = pd.concat([all_audiences, audiences], ignore_index=True)
all_audiences.to_csv(f"DataChecks/audiences/{datetime.today()}_all_audiences.csv", index=False)
all_audiences



In [15]:
all_audiences["date"] = pd.to_datetime(all_audiences["created"]).dt.date
all_audiences["week"] = pd.to_datetime(all_audiences["created"]).dt.isocalendar().week
all_audiences["year"] = pd.to_datetime(all_audiences["created"]).dt.year
all_audiences["year-week"] = all_audiences["year"].astype(str) + "-" + all_audiences["week"].astype(str)
all_audiences["week_min_date"]=all_audiences.groupby(by=["year-week"])["date"].transform("min")
all_audiences["week_max_date"]=all_audiences.groupby(by=["year-week"])["date"].transform("max")
all_audiences["week_range"] = all_audiences["week_max_date"].astype("string")+ " - " + all_audiences["week_min_date"].astype("string")
all_audiences

In [24]:
count_audiences_by_week = all_audiences.groupby(by=["year-week"])["id"].nunique().reset_index().rename(columns={"id": "new_audiences"})
get_customers_by_week = all_audiences.groupby(by=["year-week"])["workspace"].unique().reset_index()
get_audience_id_by_week = all_audiences.groupby(by=["year-week"])["id"].unique().reset_index()
count_audiences_by_week = count_audiences_by_week.merge(all_audiences.drop_duplicates(subset=["year-week"])[["year-week","week_range"]], how="left", on=["year-week"])
count_audiences_by_week = count_audiences_by_week.merge(get_customers_by_week, how="left", on=["year-week"])
count_audiences_by_week = count_audiences_by_week.merge(get_audience_id_by_week, how="left", on=["year-week"])
count_audiences_by_week = count_audiences_by_week.sort_values(by=["week_range"], ascending=False).reset_index(drop=True)
count_audiences_by_week